In [1]:
pip install boto3

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 131 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 8.6 MB/s eta 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION

In [2]:
import boto3

In [3]:
s3 = boto3.resource('s3',
aws_access_key_id='XXXXXXXXXXX',
aws_secret_access_key='XXXXXX'
)

In [6]:
try:
    s3.create_bucket(Bucket='datacont-dynamo', CreateBucketConfiguration={
'LocationConstraint': 'us-west-2'})
except Exception as e:
    print (e)

In [13]:
bucket = s3.Bucket("datacont-dynamo")
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'QVF4TXXSE73SW7A3',
  'HostId': 'X9maeBSLx8g1f85S3qTgfs0nkZ3XQ2C5atoYjxHYcdP7gX7BFHqUhjqxG2KucSaw54xbUuSUpX0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'X9maeBSLx8g1f85S3qTgfs0nkZ3XQ2C5atoYjxHYcdP7gX7BFHqUhjqxG2KucSaw54xbUuSUpX0=',
   'x-amz-request-id': 'QVF4TXXSE73SW7A3',
   'date': 'Thu, 21 Oct 2021 21:57:58 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [15]:
body = open('/Users/kish/14848/exp1.csv', 'rb')

In [18]:
o = s3.Object('datacont-dynamo', 'test').put(Body=body )


In [19]:
s3.Object('datacont-dynamo', 'test').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': '76NEFD0D94ZBNZV8',
  'HostId': 'BHd2iiDpjkYoaCP5rm43Zec+bLJgbSabMKAcWBMXduNTtueY1RdccO7DaxS+1FA9yrJCvgRuoUg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BHd2iiDpjkYoaCP5rm43Zec+bLJgbSabMKAcWBMXduNTtueY1RdccO7DaxS+1FA9yrJCvgRuoUg=',
   'x-amz-request-id': '76NEFD0D94ZBNZV8',
   'date': 'Thu, 21 Oct 2021 21:59:06 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [93]:
dyndb = boto3.resource('dynamodb',
region_name='us-west-2',
aws_access_key_id='XXXXX',
aws_secret_access_key='XXXXXX'
)


In [95]:
try:
    table = dyndb.create_table(
    TableName='DataTable4',
    KeySchema=[
    {
    'AttributeName': 'PartitionKey',
    'KeyType': 'HASH'
    },
    {
    'AttributeName': 'RowKey',
    'KeyType': 'RANGE'
    }
    ],
    AttributeDefinitions=[
    {
    'AttributeName': 'PartitionKey',
    'AttributeType': 'S'
    },
    {
    'AttributeName': 'RowKey',
    'AttributeType': 'S'
    },
    ],
    ProvisionedThroughput={
    'ReadCapacityUnits': 5,
    'WriteCapacityUnits': 5
    }
    )
except Exception as e:
    print (e)
#if there is an exception, the table may already exist. if so...
    table = dyndb.Table("DataTable3")

In [96]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [97]:
print(table.item_count)

0


In [98]:
import csv

In [99]:
with open('/Users/kish/14848/dynamoData/experiments.csv', 'r') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        # print(item)
        if(item[4] == 'URL'):
            continue
        body = open('/Users/kish/14848/dynamoData/datafiles/'+item[4],'rb')
        # body.encode('utf-8')
        s3.Object('datacont-dynamo', item[4]).put(Body=body )
        md = s3.Object('datacont-dynamo', item[4]).Acl().put(ACL='public-read')
        url = " https://s3-us-west-2.amazonaws.com/datacont-dynamo/"+item[4]
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],
        'description' : item[4], 'date' : item[2], 'url':url}
        print(metadata_item)
        try:
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

{'PartitionKey': '1', 'RowKey': '-1', 'description': 'exp1.csv', 'date': '52', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-dynamo/exp1.csv'}
{'PartitionKey': '2', 'RowKey': '-2', 'description': 'exp2.csv', 'date': '52.1', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-dynamo/exp2.csv'}
{'PartitionKey': '3', 'RowKey': '-2.93', 'description': 'exp3.csv', 'date': '57.1', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-dynamo/exp3.csv'}


In [100]:
response = table.get_item(
    Key={
        'PartitionKey': '2',
        'RowKey': '-2'
    }
)
item = response['Item']
print(response)

{'Item': {'PartitionKey': '2', 'RowKey': '-2', 'date': '52.1', 'description': 'exp2.csv', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-dynamo/exp2.csv'}, 'ResponseMetadata': {'RequestId': 'QFL41240LGTSHH5D790B9V4QP3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Thu, 21 Oct 2021 23:32:06 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '181', 'connection': 'keep-alive', 'x-amzn-requestid': 'QFL41240LGTSHH5D790B9V4QP3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '284771987'}, 'RetryAttempts': 0}}
